In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 경고 미출력 및 한글 안깨지는 코드
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system()=="Windows":
    font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

import warnings
warnings.filterwarnings("ignore")

# 연습문제1: 추천시스템

In [3]:
critics = {
    '조용필': {
        '택시운전사': 2.5,
        '겨울왕국': 3.5,
        '리빙라스베가스': 3.0,
        '넘버3': 3.5,
        '사랑과전쟁': 2.5,
        '세계대전': 3.0,
    },
    'BTS': {
        '택시운전사': 1.0,
        '겨울왕국': 4.5,
        '리빙라스베가스': 0.5,
        '넘버3': 1.5,
        '사랑과전쟁': 4.5,
        '세계대전': 5.0,
    },
    '강감찬': {
        '택시운전사': 3.0,
        '겨울왕국': 3.5,
        '리빙라스베가스': 1.5,
        '넘버3': 5.0,
        '세계대전': 3.0,
        '사랑과전쟁': 3.5,
    },
    '을지문덕': {
        '택시운전사': 2.5,
        '겨울왕국': 3.0,
        '넘버3': 3.5,
        '세계대전': 4.0,
    },
    '김유신': {
        '겨울왕국': 3.5,
        '리빙라스베가스': 3.0,
        '세계대전': 4.5,
        '넘버3': 4.0,
        '사랑과전쟁': 2.5,
    },
    '유성룡': {
        '택시운전사': 3.0,
        '겨울왕국': 4.0,
        '리빙라스베가스': 2.0,
        '넘버3': 3.0,
        '세계대전': 3.5,
        '사랑과전쟁': 2.0,
    },
    '이황': {
        '택시운전사': 3.0,
        '겨울왕국': 4.0,
        '세계대전': 3.0,
        '넘버3': 5.0,
        '사랑과전쟁': 3.5,
    },
    '이이': {'겨울왕국': 4.5, '사랑과전쟁': 1.0,
             '넘버3': 4.0},
}

In [4]:
critics

{'조용필': {'택시운전사': 2.5,
  '겨울왕국': 3.5,
  '리빙라스베가스': 3.0,
  '넘버3': 3.5,
  '사랑과전쟁': 2.5,
  '세계대전': 3.0},
 'BTS': {'택시운전사': 1.0,
  '겨울왕국': 4.5,
  '리빙라스베가스': 0.5,
  '넘버3': 1.5,
  '사랑과전쟁': 4.5,
  '세계대전': 5.0},
 '강감찬': {'택시운전사': 3.0,
  '겨울왕국': 3.5,
  '리빙라스베가스': 1.5,
  '넘버3': 5.0,
  '세계대전': 3.0,
  '사랑과전쟁': 3.5},
 '을지문덕': {'택시운전사': 2.5, '겨울왕국': 3.0, '넘버3': 3.5, '세계대전': 4.0},
 '김유신': {'겨울왕국': 3.5, '리빙라스베가스': 3.0, '세계대전': 4.5, '넘버3': 4.0, '사랑과전쟁': 2.5},
 '유성룡': {'택시운전사': 3.0,
  '겨울왕국': 4.0,
  '리빙라스베가스': 2.0,
  '넘버3': 3.0,
  '세계대전': 3.5,
  '사랑과전쟁': 2.0},
 '이황': {'택시운전사': 3.0, '겨울왕국': 4.0, '세계대전': 3.0, '넘버3': 5.0, '사랑과전쟁': 3.5},
 '이이': {'겨울왕국': 4.5, '사랑과전쟁': 1.0, '넘버3': 4.0}}

In [12]:
# 영화 목록 구하기

movieList = []
for name in critics:
    for movie in critics[name]:
        if movie in movieList:
            pass
        else:
            movieList.append(movie)
movieList

['택시운전사', '겨울왕국', '리빙라스베가스', '넘버3', '사랑과전쟁', '세계대전']

In [10]:
# 사람 사이 유사도 구하기

def sim_pearson(data, name1, name2):
    sumx = 0    # X의 합
    sumy = 0    # Y의 합
    sumxy = 0    # XY의 합
    sumx2 = 0    # X 제곱의 합
    sumy2 = 0    # Y 제곱의 합
    n = 0    # 영화 개수
    for movie in data[name1]:
        if movie in data[name2]:
            sumx += data[name1][movie]
            sumy += data[name2][movie]
            sumxy += data[name1][movie] * data[name2][movie]
            sumx2 += pow(data[name1][movie], 2)
            sumy2 += pow(data[name2][movie], 2)
            n += 1
    return (sumxy - ((sumx * sumy) / n)) / sqrt((sumx2 - (pow(sumx,2) / n)) * (sumy2 - (pow(sumy,2) / n)))

In [24]:
# 영화 추천 함수

def movieRec(data, name):
    recList = []
    
    for movie in movieList:
        sumPredicScore = 0
        sumSimilarity = 0
        if movie not in data[name]:    # 타겟 사람이 안 본 영화라면
            for other in data:
                if other != name:    # 목록에 있는 다른 사람들 중에
                    if movie in data[other]:    # 해당 영화를 봤다면
                        sim = sim_pearson(data, name, other)    # 두 사람의 유사도
                        predicScore = sim * data[other][movie]    # 타겟 예상점수 = 유사도 * 다른사람 평점
                        sumPredicScore += predicScore
                        sumSimilarity += sim
            recList.append((sumPredicScore/sumSimilarity, movie))    # (예상점수 총합, 영화 제목)
    recList.sort()
    recList.reverse()
    return recList[0][1]

In [25]:
movieRec(critics, "이이")

'택시운전사'